In [29]:
import pandas as pd
import numpy as np
import os
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [30]:
print(os.getcwd())

/root/TSTF/torch


`-` All ens

In [6]:
mse_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_mse_pred.csv", index_col = 0)).reshape(2, -1))
mse_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_mse_pred.csv", index_col = 0)).reshape(2, -1))

mae_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_mae_pred.csv", index_col = 0)).reshape(2, -1))
mae_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_mae_pred.csv", index_col = 0)).reshape(2, -1))

mase_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_mase_pred.csv", index_col = 0)).reshape(2, -1))
mase_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_mase_pred.csv", index_col = 0)).reshape(2, -1))

mape_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_mape_pred.csv", index_col = 0)).reshape(2, -1))
mape_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_mape_pred.csv", index_col = 0)).reshape(2, -1))

smape_pred = list(np.array(pd.read_csv("~/TSTF/main/resulttf/val/trTFTF_coin_smape_pred.csv", index_col = 0)).reshape(2, -1))
smape_pred2 = list(np.array(pd.read_csv("~/TSTF/main/resulttf/test/trTFTF_coin_smape_pred.csv", index_col = 0)).reshape(2, -1))

In [7]:
data = "coin"
model_name = 'trTFMLP'
num = 100   ## model k per loss

In [8]:
## target domain
target_X = pd.read_csv(f"../data/{data}/train_input_7.csv").iloc[:, 1:].values.astype(np.float32)
target_y = pd.read_csv(f"../data/{data}/train_output_7.csv").iloc[:, 1:].values.astype(np.float32)

target_X_val = target_X[-round(target_X.shape[0] * 0.2):, :].astype(np.float32)
target_y_val = target_y[-round(target_y.shape[0] * 0.2):].astype(np.float32)
target_X = target_X[:-round(target_X.shape[0] * 0.2), :].astype(np.float32)
target_y = target_y[:-round(target_y.shape[0] * 0.2)].astype(np.float32)

test_X  = pd.read_csv(f"../data/{data}/val_input_7.csv").iloc[:, 1:].values.astype(np.float32)
test_y  = pd.read_csv(f"../data/{data}/val_output_7.csv").iloc[:, 1:].values.astype(np.float32)

In [9]:
val_preds = {}
test_preds = {}
save_name = ["mse", "mae", "mase", "mape", "smape"]

val_preds["mse"] = mse_pred
val_preds["mae"] = mae_pred
val_preds["MASE"] = mase_pred
val_preds["mape"] = mape_pred
val_preds["SMAPE"] = smape_pred

In [18]:
concat_G = np.concatenate(list(val_preds.values()))
fin_pred_G = np.median(concat_G, axis = 0)
print("all (RMSE):", np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))
print(f"all (MAE):  {mean_absolute_error(target_y_val.flatten(), fin_pred_G.flatten()):.5f}")

## 어떤 점에서 best?
concat_G = np.concatenate([val_preds[loss] for loss in ["mae", "MASE", "mse"]])
fin_pred_G = np.median(concat_G, axis = 0)
print("best (RMSE):", np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))
print(f"best (MAE):  {mean_absolute_error(target_y_val.flatten(), fin_pred_G.flatten()):.5f}")

## 변수별 앙상블 결과
for name in ["mse", "mae", "MASE", "mape", "SMAPE"]:
    concat_G = np.concatenate([np.nan_to_num(np.array(val_preds[name]), nan=0)])
    fin_pred_G = np.median(concat_G, axis=0)
    print(name, "(RMSE):", np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred_G.flatten())).round(5))
    print(name, f"(MAE):  {mean_absolute_error(target_y_val.flatten(), fin_pred_G.flatten()):.5f}")

all (RMSE): 29.31602
all (MAE):  21.50396
best (RMSE): 22.71764
best (MAE):  16.05210
mse (RMSE): 30.82467
mse (MAE):  24.65280
mae (RMSE): 25.1985
mae (MAE):  19.25673
MASE (RMSE): 26.72937
MASE (MAE):  20.66004
mape (RMSE): 69.82386
mape (MAE):  67.14908
SMAPE (RMSE): 55.74698
SMAPE (MAE):  43.22006


In [21]:
performance = np.array([MASE_rmse, MAPE_rmse, sMAPE_rmse, MAE_rmse, MSE_rmse])

In [ ]:
performance = np.array([MASE_rmse, MAPE_rmse, sMAPE_rmse, MAE_rmse, MSE_rmse])
os.makedirs('resulttf', exist_ok=True)
pd.DataFrame(performance).to_csv(f"resulttf/trTFTF_{data}_weight.csv")

In [ ]:
rmse = []
mae = []

for name in ["MASE", "mape", "SMAPE", "mae", "mse"]:
    concat_G = np.concatenate([np.nan_to_num(np.array(val_preds[name]), nan = 0)])
    fin_pred = np.median(concat_G, axis = 0)
    rmse.append(np.sqrt(mean_squared_error(target_y_val.flatten(), fin_pred.flatten())).round(5))
    mae.append(round(mean_absolute_error(target_y_val.flatten(), fin_pred.flatten()), 5))

In [ ]:
performance = np.array(rmse)

In [27]:
performance

array([26.72937, 69.82386, 55.74698, 25.1985 , 30.82467])